In [1]:
import pandas as pd
import numpy as np
import sys
import os
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

ASIM_OUTPUT = r"E:\Met_Council\metc-asim-model\estimation\output"
ASIM_CONFIGS_PATH = r'E:\Met_Council\metc-asim-model\source\activitysim\configs'
SURVEY_SUMMARY_PATH = r'E:\Met_Council\survey_data\Phase1\obs_processing\output'
SYNPOP_INPUT = r'E:\Met_Council\metc-asim-model\Input\socioeconomic'
SURVEY_PATH = r'E:\Met_Council\survey_data\Phase1\SPA_Inputs'
CENSUS_SUMMARY_PATH = r'E:\Met_Council\metc-asim-model\Input\Visualizer\census'

census = pd.read_csv(os.path.join(CENSUS_SUMMARY_PATH, 'census_acs_2018_wrk_auto.csv')).rename(columns = {'num_workers': 'HH_WORKERS_CAT',
	'auto_ownership': 'HH_VEH_CAT'}).groupby(['HH_WORKERS_CAT', 'HH_VEH_CAT']).agg(census_hh = ('value', 'sum'))

survey = pd.read_csv(os.path.join(SURVEY_PATH, 'HH_SPA_INPUT.csv')).assign(HH_VEH_CAT = lambda x: x.HH_VEH_CAT.clip(upper = 3)).groupby(['HH_WORKERS_CAT', 'HH_VEH_CAT']).agg(survey_hh = ('HHEXPFAC', sum))

model_hh = pd.read_csv(os.path.join(ASIM_OUTPUT, 'final_households.csv'))

DAMPING = 0.75

MODEL_EXPANSION = model_hh.shape[0] / survey['survey_hh'].sum()
print(f"Expanding the model output by {MODEL_EXPANSION}")

model_hh = model_hh.assign(num_workers = lambda x: x.num_workers.clip(upper = 3), auto_ownership = lambda x: x.auto_ownership.clip(upper = 3)).rename(columns = {'num_workers': 'HH_WORKERS_CAT', 'auto_ownership': 'HH_VEH_CAT'}).groupby(['HH_WORKERS_CAT', 'HH_VEH_CAT']).agg(model_hh = ('household_id', 'count')) / MODEL_EXPANSION

# valid_table = model_hh.join(survey)
valid_table = model_hh.join(census)

asim_coefs = pd.read_csv(os.path.join(ASIM_CONFIGS_PATH, 'auto_ownership_coefficients.csv')).set_index('coefficient_name')

valid_table.style.format(formatter = {'model_hh': '{:,.0f}', 'survey_hh': '{:,.0f}', 'census_hh': '{:,.0f}'})

Expanding the model output by 0.028687048508594413


In [2]:
coef_table = pd.DataFrame({'HH_WORKERS_CAT': [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2], 'HH_VEH_CAT': [0, 0, 0, 2, 2, 2, 3, 3, 3, 1, 1],
    'coefficient_name': ['coef_cars0_workers_1', 'coef_cars0_workers_2', 'coef_cars0_workers_3plus', 'coef_cars2_workers_1', 'coef_cars2_workers_2', 
    'coef_cars2_workers_3plus', 'coef_cars34_workers_1', 'coef_cars34_workers_2', 'coef_cars34_workers_3plus', 'coef_cars1_workers_1',
    'coef_cars1_workers_2'
    ]}).set_index(['HH_WORKERS_CAT', 'HH_VEH_CAT'])

coef_adj_table = valid_table.join(coef_table).merge(asim_coefs.rename(columns = {'value': 'init_value'}), how = 'inner', left_on = 'coefficient_name', right_index = True)
# coef_adj_table['adjust'] = np.log(coef_adj_table['survey_hh'] / coef_adj_table['model_hh']) * DAMPING
coef_adj_table['adjust'] = np.log(coef_adj_table['census_hh'] / coef_adj_table['model_hh']) * DAMPING
coef_adj_table.loc[coef_adj_table['constrain'] == 'T', 'adjust'] = 0
coef_adj_table['value'] = coef_adj_table['init_value'] + coef_adj_table['adjust']

coef_adjustments = coef_adj_table.reset_index().set_index('coefficient_name')

coef_adjustments.to_excel("adj_check.xlsx")
coef_adjustments.style.format(formatter = {'model_hh': '{:,.0f}', 'survey_hh': '{:,.0f}', 'census_hh': '{:,.0f}', 'init_value': '{:0.04f}', 'value': '{:0.04f}'})


,HH_WORKERS_CAT,HH_VEH_CAT,model_hh,census_hh,init_value,constrain,adjust,value
coefficient_name,,,,,,,,
coef_cars0_workers_1,1,0,"31,338","30,353",-1.2315,F,-0.023956,-1.2554
coef_cars1_workers_1,1,1,"303,168","244,035",-0.7222,F,-0.162732,-0.8850
coef_cars2_workers_1,1,2,"286,087","166,414",0.0000,T,0.000000,0.0000
coef_cars34_workers_1,1,3,"4,950","47,039",-2.5007,F,1.688697,-0.8120
coef_cars0_workers_2,2,0,"8,645","7,831",-1.4583,F,-0.074170,-1.5325
coef_cars1_workers_2,2,1,"42,284","40,538",-2.3965,F,-0.031625,-2.4281
coef_cars2_workers_2,2,2,"335,273","321,321",-1.8475,F,-0.031879,-1.8793
coef_cars34_workers_2,2,3,"44,585","106,990",-2.7501,F,0.656510,-2.0936
coef_cars0_workers_3plus,3,0,314,"1,860",-1.1341,F,1.334848,0.2008


In [3]:
# ⚠ This will write to the coefficients table!
asim_coefs.update(coef_adjustments['value'])

asim_coefs.reset_index().to_csv(os.path.join(ASIM_CONFIGS_PATH, 'auto_ownership_coefficients.csv'), index = False)